In [10]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import scipy.signal as signal
import rasterio
import scipy.stats as stats
from scipy.stats import hypergeom
from scipy.interpolate import RegularGridInterpolator
from matplotlib.colors import LinearSegmentedColormap
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import ListedColormap
import matplotlib.colors as mcolors
from tqdm import tqdm
import geopandas as gpd
from pathlib import Path
import pickle

In [3]:
# --path--
# era5land_quarterly_data = "/vast/bzaitch1/trp_climate_model_data/era5land_1970_2024_qtrmean" # Location on Rockfish
# output_directory  = "/vast/bzaitch1/trp_climate_model_data/out"
# os.makedirs(output_directory, exist_ok=True)
# quarter_dir = Path(rf"E:\backup\trp_climate_model_data\era5land_1970_2024_qtrmean")

quarter_dir = Path("/Users/kris/Local Job Backup/test set/") # Location on local machine
world_shp = gpd.read_file('data/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp')
test_data_set = quarter_dir / '*.nc'
years = range(2001, 2020)
quarters = range(1, 5)
pattern_per_month = "{y}_Q{q}_qmean.nc"   # for one file per month
grid_data_list =[]

# Hourly filename pattern:
# One file per month:    YYYY_MM.nc
for year in years:
    for quarter in quarters:
        data_file_name = quarter_dir / pattern_per_month.format(y=year, q=quarter)
        grid_data_list.append(data_file_name)


cvar_aliases = {
    'tp' : 'total precipitation', 
    't2m' : '2 metre temperature',
    'e' : 'evaporation',
}

evar_aliases = {
    'y' : 'Log of real GDP', 
    'Dp' : 'Inflation rate (Based on CPI)',
    'eq' : 'Log of real equity prices',
    'ep' : 'Log of real exchange rate',
    'r' : 'Short-term interest rate',
    'lr' : 'Long-term interest rate',
}

country_study_sites = ['India', 'Brazil', 
                       'Chile', 'Indonesia', 
                       'Mexico', 'Peru', 
                       'Philippines', 'South Africa', 
                       'Thailand', 'Kenya',
                       'Egypt', 'Columbia']

In [7]:
instance_ds = xr.open_dataset(grid_data_list[1])
#instance_ds

lats = instance_ds['latitude'].values
lons = instance_ds['longitude'].values
lons = np.where(lons > 180, lons - 360, lons) # same as SPEI and crop
ds = instance_ds.assign_coords(longitude=lons)

# Create meshgrid of lat/lon pairs
lon_grid, lat_grid = np.meshgrid(lons, lats)
coordinates = np.column_stack([lon_grid.ravel(), lat_grid.ravel()])

In [ ]:
# --- fix longitude from 0..360 to -180..180 and sort ---
if float(da.longitude.max()) > 180:
    lon_new = ((da.longitude + 180) % 360) - 180
    da = da.assign_coords(longitude=lon_new).sortby("longitude")

# --- make latitude ascending (optional but helpful for weights) ---
if da.latitude[0] > da.latitude[-1]:
    da = da.sortby("latitude")

array([ 0. ,  0.1,  0.2, ..., -0.3, -0.2, -0.1])

In [14]:
world = gpd.read_file('data/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp')

mask_dict = {}
points_in_countries_all = {}
#points_idx_climate = {}

for country_name in tqdm(country_study_sites):
    country = world[world['SOVEREIGNT'] == country_name]
    # Create a GeoDataFrame with the mesh grid points
    points_gdf = gpd.GeoDataFrame(pd.DataFrame(coordinates, columns=['Longitude', 'Latitude']),
                                geometry=gpd.points_from_xy(coordinates[:, 0], coordinates[:, 1]), crs="EPSG:4326" )
    
    points_in_countries =  gpd.sjoin(points_gdf, country, predicate='within')
    points_in_countries_all[country_name] = points_in_countries
    # mask = np.zeros(lon_grid.shape, dtype=bool)

    # indices = points_in_countries.index.values
    # mask[np.unravel_index(indices, lon_grid.shape)] = True
    # mask_dict[country_name] = mask
    
    # index = []
    # for lon, lat in points_in_countries[['Longitude', 'Latitude']].values:
    #     # Find the index of the closest point
    #     lat_idx = abs(ds['latitude'] - lat).argmin().item()
    #     lon_idx = abs(ds['longitude'] - lon).argmin().item()
    #     index.append([lat_idx, lon_idx])
    # points_idx_climate[country_name] = index

100%|██████████| 12/12 [01:03<00:00,  5.33s/it]


In [22]:
# Save data to a pickle file
with open("data/lon_lat_12c.pkl", "wb") as file:
    pickle.dump(points_in_countries_all, file)

with open("data/lon_lat_12c.pkl", "rb") as file:
    points_in_countries_all = pickle.load(file)

In [25]:
points_in_countries_all['Brazil'].index.values

array([3055793, 3055794, 3055796, ..., 4452667, 4452668, 4456266])